# **Competencia 1 - CC6205 Natural Language Processing 📚**

**Integrantes: Vicente Ardiles y Rodrigo Oportot**

**Usuario del equipo en CodaLab: NLPachi**

**Fecha límite de entrega 📆:** Miércoles 12 de mayo.

**Tiempo estimado de dedicación: 25hrs**

# **Entregable.**

## **1. Introducción**

De manera general, el problema a resolver consiste en clasificar la intensidad de un sentimiento relacionado a un tweet mediante el uso de herramientas asociadas a la disciplina de NLP. Una formalización concreta de la *task* en cuestión es:

- Clasificar la intensidad de un sentimiento en un conjunto de tweets conociendo dicho sentimiento de antemano.

La task pertenece al mundo de clasificación de texto. 

Los inputs del problema tienen la siguiente estructura: 



In [112]:
train['anger'].sample(1)

,id,tweet,sentiment,sentiment_intensity
576,10576,"@ticcikasie1 With a frown, she let's out a dis...",anger,medium


El archivo que contiene los datos está en formato CSV y es leído utilizando la librería Pandas, en donde se encuentran 4 columnas indicando el id del tweet, su contenido, el sentimiento asociado y la intensidad de este último. En particular, hay cuatro sentimientos y tres intensidades para cada uno, siendo:

- Sentimientos: **anger, fear, joy y sadness**
- Intensidad: **low, medium y high**

La pipeline encargada de resolver el problema toma el set de entrenamiento y luego el de testeo (también hay casos donde hubo set de entrenamiento, validación y finalmente testeo). Para entrenar al clasificador, esta estructura se apoya en un tokenizador, un vectorizador y en varias features descritas en las secciones posteriores del reporte. Finalmente, se obtienen los valores para cada métrica de análisis sobre el nuevo etiquetado producido por el clasificador.

A primera vista, el corpus a trabajar presenta ciertos desafíos como la manera de manejar los emoticones, ya que esto puede significar muchas cosas: contarlos, catalogarlos y asociarlos a ciertos sentimientos, asignarles una ponderación para representar tal sentimiento, etc. Por otro lado, está el tratamiento de los hashtags "#", pudiendo estos aportar de manera concreta al cálculo de clasificación de intensidad en base a su contenido.

Por un lado de carácter subyacente, otras dificultades por afrontar contemplan la creación de features especiales para la task, así como la elección particular de ellas al momento de experimentar con la pipeline, buscando una combinación adecuada. A causa de la maldición de la dimensionalidad, colocar todas las features de una vez en un solo experimento (incluso con reducción, por ejemplo con PCA), podría resultar contraproducente, entorpeciendo la correcta clasificación descrita en la task. 

Para finalizar este apartado, la metodología de desarrollo consiste en programar las features en coherencia con el corpus y la task, pre-procesar el corpus tokenizando de manera personalizada apoyándose en vectorizadores de sklearn y los distintos módulos de nltk y, finalmente, tomar tres algoritmos de clasificación (clasificadores Naive Bayes, SVM y Regresión Logística) para introducir en la pipeline, con el fin de analizar el desempeño del conjunto elegido de herramientas en base a los valores que tomen las métricas de evaluación descritas más adelante.

## **2. Representaciones**

Para representar los tweets de manera numérica, se utilizaron dos tipos de *vectorizers*: CountVectorizer y TfidfVectorizer, convirtiendo a matrices numéricas los tweets, junto con un tokenizador personalizado que los pre-procesa mediante:

- TweetTokenizer(strip_handles=True, reduce_len=True), proveniente de la librería nltk, encargado de tokenizar eliminando símbolos como @ y símbolos ([. ! ?] entre otros) repetidos más de tres veces.

- mark_negation() de nltk, que marca las palabras después de una negación con un tag "_Neg", para conservar su contexto al quitar stopwords posteriormente.

- remStopWords() con las stopwords de nlkt, removiendo las stopwords.

- hashtagToWord(), que quita el símbolo # de un hashtag, dejando solamente la palabra (esta decisión se explicará más adelante, pero en resumen, se obtienen mejores resultados de esta manera).

- stemmize() con SnowballStemmer de nltk.

- lemmatize() con WordNetLemmatizer de nltk.

Como se mencionó anteriormente, todas estas funciones son parte de my_tokenizer(text), añadida como parámetro a los vectorizers por utilizar. 

Se crearon y utilizaron diversas features mediante los transformers de sklearn:

- DotsMarksCountTransformer, que cuenta en un tweet la cantidad de veces que los símbolos [. ! ?] se repiten, al menos, 3 veces consecutivamente.

- UppercaseCountTransformer, contando la cantidad de palabras que están completamente en mayúsculas y son de largo mayor a 1.

- ElongatedWordsCountTransformer, contando la cantidad de palabras que tienen al menos 3 letras repetidas de manera consecutiva.

- NegationCountTransformer, contando la cantidad de palabras con la etiqueta "_NEG" a causa de pre-procesar el tweet con mark_negation() de nltk.

- CharsCountTransformer, la feature que venía con el código base, contando el número de símbolos [# ! ? @] en u tweet.

- LexiconCountTransformer(emotion), feature más compleja que calcula el número de lexicons asociado al tipo de sentimiento de un tweet, junto con calcular el promedio del valor que representa la intensidad de dicha emoción. Por ejemplo, si un tweet proveniente del dataset angry tuviera un lexicon A asociado a angry con una ponderación de 0.8 en relación a la intensidad, y también tuviera un lexicon B asociado a angry con una intensidad de 0.2, la feature entregaría [2 (0.8 + 0.2)/2] como resultado. Dado que hay valores negativos para la intensidad de emoción en los lexicons (extraídos de SenticNet5), se toma el valor absoluto de ellos. 

- EmojiCount, contando la cantidad de emojis por tweet

- Doc2VecTransformer, feature analogada del auxiliar de words embedding para extraer las mismas de los tweets a analizar, trabajando con la librería gensim.



## **3. Algoritmos**

Se utilizaron los algoritmos de Naive Bayes, Support Vector Machine (SVM) lineal y Logistic Regression. 

Naive Bayes (MultinomialNB) es un algoritmo basado en el teorema de Bayes,  que asume independencia entre las features asociadas al momento de clasificar, simplificando los cálculos. Es por esta razón que no es posible utilizar features de n-gramas en este caso, ya que rompería con la suposición inicial. Cabe destacar que, si se tomara un enfoque más detallista, la suposición de independencia no sería cierta dependiendo del corpus por analizar.

Este algoritmo emplea el teorema de Bayes para asignarle a cada documento del corpus la probabilidad de pertenecer a una clase dada. Sin entrar en detalles matemáticos, el algoritmo revisa los componentes tokenizados, viendo la probabilidad de que cada palabra pertenezca o no a cada clase. No importa el orden de palabras en este algoritmo, pudiendo usar como tokenizador una simple bag of words (en el caso más sencillo). 

El segundo clasificador utilizado fue SVM lineal. Matemáticamente es complejo de detallar, pero de manera llana, se encarga de buscar el hiperplano óptimo para separar los conjuntos de datos en las clases por clasificar dentro de $\mathbb{R}^{2}$.

Por último, Logistic Regression es un modelo de clasificación lineal donde las probabilidades para un posible resultado son modelas usando una función logística. Dada su complejidad, tampoco se entrará en detalle su funcionamiento específico. Sin embargo, se puede especificar que los resultados dependen de variables que se modelan binariamente. Lo anterior implica varios supuestos, como el hecho de que las variables sean binarias, sumado a que deben ser linealmente independientes entre sí. En este caso, se usó la extensión multinomial del clasificador para predecir la intensidad del sentimiento.

## **4. Métricas de Evaluación**

Para evaluar el desempeño de la solución, se utilizan las siguientes métricas:

- AUC: El área bajo la curva ROC, donde toma el valor 1 si es un clasificador perfecto y 0.5 si es uno aleatorio. La curva ROC grafica la proporción de verdaderos positivos (recall) versus la proporción de falsos positivos. 

- Kappa: En casos donde una task de clasificación deba trabajar con datos desbalanceados en favor de una clase mayoritaria, esta métrica trata esto normalizando la accuracy de clasificación en base a tal desbalance. Un clasificador que siempre está correcto tendrá Kappa igual a 1, mientras que un clasificador que acierte con la misma probabilidad que uno aleatorio, tendrá un valor igual a 0.

- Accuracy: Cantidad de documentos clasificados correctamente vs la cantidad total de documentos.


## **5. Diseño experimental**

### **5.1. Diseños de prueba y ensayos**

Previo a comentar la metodología experimental, es necesario destacar los ensayos de prueba que permitieron diseñar los experimentos finales. Los resultados de éstos fueron analizados en el momento, de tal manera que sirvieran como una guía que encaminara el desarrollo experimental.

- En primera instancia se probó con Oversampling solamente sobre las muestras de training, pero más adelante se descartó ya que se obtuvieron clasificadores overfitteados para todos los modelos, con resultados sumamente optimista por todas las métricas de evaluación. Se notó que las particiones de training y testeo seguían desbalanceadas cuando el clasificador las recibía con fit. Para solucionar esto, se volvió a trabajar con el dataset original desbalanceado, bajo la premisa de que, para la clasificación, es más sencillo catalogar los casos extremos (high y low) que los de medium, decidiendo transmitir ese sesgo a través del modelo, ya que un sentimiento neutro parece ser más difícil de detectar, a pesar de tener más muestras de él. La premisa es cuestionable, obviamente, pero tomarla como referencia ayudó a obtener resultados posteriores mucho mejores que en un comienzo. Esto es algo que podría volver a discutirse en el futuro. 

- A su vez, el modelo MultinomialNB fue descartado posteriormente (a pesar de experimentar con él), ya que sus resultados eran inferiores a los producidos por SVM y Logistic Regression (LR por abreviar). 

- Las features de UpperCase, ElongatedWords y EmojiCount fueron descartadas ya que empeoraban el desempeño de los clasificadores en todos los casos.

- Se decidió no utilizar la función que remueve stopwords en el tokenizador, ya que después de varios ensayos, se notó que afectaba negativamente el desempeño de los clasificadores.

- La feature de Lexicons a veces arrojaba errores con otras features. Fue un problema difícil de abordar, por lo que se decidió, en mayor medida, descartar.

### 5.2. Diseño experimental



La metodología experimental consistió en tomar variables de control en las pipelines construidas, para comparar diversos modelos, tokenizers y features a la hora de entrenar los modelos. 

Cabe destacar que los siguientes experimentos fueron realizados con el mismo tokenizador *my_tokenizer(text)*, con las funciones descritas en el apartado 2 de Representaciones, expecto la de stopwords, ya que arrojaba resultados inferiores en los intentos de prueba.

- Experimento 1:
Modelo LR con TfidfCountVectorizer(tokenizer=my_tokenizer) y las features chars_count, NegationCount, WordEmbeddings. 

- Experimento 2:
Modelo LR con CountVectorizer(ngram=(1,3), max_features=9000, tokenizer=my_tokenizer) y las features chars_count, NegationCount, WordEmbeddings.

- Experimento 3:
Modelo SVC con CountVectorizer(ngram=(1,3), max_features=9000, tokenizer=my_tokenizer) y las features chars_count, NegationCount, WordEmbeddings.

- Experimento 4:
Modelo SVC con TfidfCountVectorizer(tokenizer=my_tokenizer) y las features chars_count, NegationCount, WordEmbeddings.

- Experimento 5:
Modelo LR con TfidfCountVectorizer(tokenizer=my_tokenizer) y las features chars_count, NegationCount, WordEmbeddings y Lexicons.

## **6. Experimentos**

### Importar librerías

In [ ]:
import pandas as pd
import os
import numpy as np
import shutil

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import confusion_matrix, cohen_kappa_score, classification_report, accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin


# word2vec
from gensim.models import Word2Vec, KeyedVectors
from gensim.models.phrases import Phrases, Phraser

import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

### Definir métodos de evaluación (**NO tocar este código**)

Estas funciones están a cargo de evaluar los resultados de la tarea. No deberían cambiarlas.


In [ ]:
def auc_score(test_set, predicted_set):
    high_predicted = np.array([prediction[2] for prediction in predicted_set])
    medium_predicted = np.array(
        [prediction[1] for prediction in predicted_set])
    low_predicted = np.array([prediction[0] for prediction in predicted_set])
    high_test = np.where(test_set == 'high', 1.0, 0.0)
    medium_test = np.where(test_set == 'medium', 1.0, 0.0)
    low_test = np.where(test_set == 'low', 1.0, 0.0)
    auc_high = roc_auc_score(high_test, high_predicted)
    auc_med = roc_auc_score(medium_test, medium_predicted)
    auc_low = roc_auc_score(low_test, low_predicted)
    auc_w = (low_test.sum() * auc_low + medium_test.sum() * auc_med +
             high_test.sum() * auc_high) / (
                 low_test.sum() + medium_test.sum() + high_test.sum())
    return auc_w


def evaluate(predicted_probabilities, y_test, labels, dataset_name):
    # Importante: al transformar los arreglos de probabilidad a clases,
    # entregar el arreglo de clases aprendido por el clasificador.
    # (que comunmente, es distinto a ['low', 'medium', 'high'])
    predicted_labels = [
        labels[np.argmax(item)] for item in predicted_probabilities
    ]


    print('Confusion Matrix for {}:\n'.format(dataset_name))
    print(
        confusion_matrix(y_test,
                         predicted_labels,
                         labels=['low', 'medium', 'high']))

    print('\nClassification Report:\n')
    print(
        classification_report(y_test,
                              predicted_labels,
                              labels=['low', 'medium', 'high']))

    # Reorder predicted probabilities array.
    labels = labels.tolist()
    
    predicted_probabilities = predicted_probabilities[:, [
        labels.index('low'),
        labels.index('medium'),
        labels.index('high')
    ]]
    
    
    auc = round(auc_score(y_test, predicted_probabilities), 3)
    #print("Scores:\n\nAUC: ", auc, end='\t')
    kappa = round(cohen_kappa_score(y_test, predicted_labels), 3)
    #print("Kappa:", kappa, end='\t')
    accuracy = round(accuracy_score(y_test, predicted_labels), 3)
    #print("Accuracy:", accuracy)
    #print('------------------------------------------------------\n')
    return np.array([auc, kappa, accuracy])

### Datos



In [ ]:
# Datasets de entrenamiento.
train = {
    'anger': pd.read_csv('https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/assignment_1/data/train/anger-train.txt', sep='\t', names=['id', 'tweet', 'class', 'sentiment_intensity']),
    'fear': pd.read_csv('https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/assignment_1/data/train/fear-train.txt', sep='\t', names=['id', 'tweet', 'class', 'sentiment_intensity']),
    'joy': pd.read_csv('https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/assignment_1/data/train/joy-train.txt', sep='\t', names=['id', 'tweet', 'class', 'sentiment_intensity']),
    'sadness': pd.read_csv('https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/assignment_1/data/train/sadness-train.txt', sep='\t', names=['id', 'tweet', 'class', 'sentiment_intensity'])
}

for key, value in train.items():
    value.rename(columns={'class': 'sentiment'}, inplace=True)

# Datasets que deberán predecir para la competencia.
target = {
    'anger': pd.read_csv('https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/assignment_1/data/target/anger-target.txt', sep='\t', names=['id', 'tweet', 'class', 'sentiment_intensity'], na_values=['NONE']),
    'fear': pd.read_csv('https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/assignment_1/data/target/fear-target.txt', sep='\t', names=['id', 'tweet', 'class', 'sentiment_intensity'], na_values=['NONE']),
    'joy': pd.read_csv('https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/assignment_1/data/target/joy-target.txt', sep='\t', names=['id', 'tweet', 'class', 'sentiment_intensity'], na_values=['NONE']),
    'sadness': pd.read_csv('https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/assignment_1/data/target/sadness-target.txt', sep='\t', names=['id', 'tweet', 'class', 'sentiment_intensity'], na_values=['NONE'])
}

for key, value in target.items():
    value.rename(columns={'class': 'sentiment'}, inplace=True)

In [ ]:
# Ejemplo de algunas filas aleatorias del dataset etiquetado:
train['anger'].sample(5)

,id,tweet,sentiment,sentiment_intensity
375,10375,@cc_yandian @HillaryClinton her team must draw...,anger,medium
449,10449,Hey folks sorry if anything offensive got post...,anger,medium
302,10302,ESPN just assumed I wanted their free magazine...,anger,medium
216,10216,I don't talk about politics because people now...,anger,medium
363,10363,@laura221b I've left it for my dad to deal wit...,anger,medium


In [ ]:
# Ejemplo de algunas filas aleatorias del dataset no etiquetado
target['anger'].sample(5)

,id,tweet,sentiment,sentiment_intensity
391,11332,@RadioX @ChrisMoyles wow. not heard this in fo...,anger,NaN
268,11209,i resent being tired and annoyed that the word...,anger,NaN
233,11174,@thehill no wonder USA is going to shit with s...,anger,NaN
647,11588,"And Republicans, you, namely Graham, Flake, Sa...",anger,NaN
497,11438,@victoriarghhh not mad but tilting? slightly i...,anger,NaN


### Analizar los datos 

En esta sección analizaremos el balance de los datos. Para ello se imprime la cantidad de tweets de cada dataset agrupados por la intensidad de sentimiento.

In [ ]:
for dataset_name in train:
    print(f'Dataset: {dataset_name} \n', train[dataset_name].groupby(['sentiment_intensity']).size())
    print('----------------------------------------------------------\n')

Dataset: anger 
 sentiment_intensity
high      163
low       161
medium    617
dtype: int64
----------------------------------------------------------

Dataset: fear 
 sentiment_intensity
high      270
low       288
medium    699
dtype: int64
----------------------------------------------------------

Dataset: joy 
 sentiment_intensity
high      195
low       219
medium    488
dtype: int64
----------------------------------------------------------

Dataset: sadness 
 sentiment_intensity
high      197
low       210
medium    453
dtype: int64
----------------------------------------------------------



Undersampling que se almacena en una nueva variable en caso de ser usado

In [ ]:
#Under-sampling
#anger
under_anger = pd.DataFrame(train['anger'])
MIN_ANGER = np.min(under_anger.sentiment_intensity.value_counts().values)

a = train['anger'].groupby('sentiment_intensity')
under_anger = pd.DataFrame(a.apply(lambda x: x.sample(MIN_ANGER).reset_index(drop=True))).reset_index(
        drop=True)
under_anger.sentiment_intensity.value_counts()

#fear
under_fear = pd.DataFrame(train['fear'])
MIN_FEAR = np.min(under_fear.sentiment_intensity.value_counts().values)

a = train['fear'].groupby('sentiment_intensity')
under_fear = pd.DataFrame(a.apply(lambda x: x.sample(MIN_FEAR).reset_index(drop=True))).reset_index(
        drop=True)
under_fear.sentiment_intensity.value_counts()

#joy
under_joy = pd.DataFrame(train['joy'])
MIN_JOY = np.min(under_joy.sentiment_intensity.value_counts().values)

a = train['joy'].groupby('sentiment_intensity')
under_joy = pd.DataFrame(a.apply(lambda x: x.sample(MIN_JOY).reset_index(drop=True))).reset_index(
        drop=True)
under_joy.sentiment_intensity.value_counts()

#sadness
under_sadness = pd.DataFrame(train['sadness'])
MIN_SADNESS = np.min(under_sadness.sentiment_intensity.value_counts().values)

a = train['sadness'].groupby('sentiment_intensity')
under_sadness = pd.DataFrame(a.apply(lambda x: x.sample(MIN_SADNESS).reset_index(drop=True))).reset_index(
        drop=True)
under_sadness.sentiment_intensity.value_counts()

train_undersampling = {
    'anger': under_anger,
    'fear': under_fear,
    'joy': under_joy,
    'sadness': under_sadness
}

Oversampling

In [ ]:
#Over-sampling
#anger
over_anger = pd.DataFrame(train['anger'])
MAX_ANGER = np.max(over_anger.sentiment_intensity.value_counts().values)

a = train['anger'].groupby('sentiment_intensity')
over_anger = pd.DataFrame(a.apply(lambda x: x.sample(MAX_ANGER, replace=True).reset_index(drop=True))).reset_index(
        drop=True)

#fear
over_fear = pd.DataFrame(train['fear'])
MAX_FEAR = np.max(over_fear.sentiment_intensity.value_counts().values)

a = train['fear'].groupby('sentiment_intensity')
over_fear = pd.DataFrame(a.apply(lambda x: x.sample(MAX_FEAR, replace=True).reset_index(drop=True))).reset_index(
        drop=True)

#joy
over_joy = pd.DataFrame(train['joy'])
MAX_JOY = np.max(over_joy.sentiment_intensity.value_counts().values)

a = train['joy'].groupby('sentiment_intensity')
over_joy = pd.DataFrame(a.apply(lambda x: x.sample(MAX_JOY, replace=True).reset_index(drop=True))).reset_index(
        drop=True)

#sadness
over_sadness = pd.DataFrame(train['sadness'])
MAX_SADNESS = np.max(over_sadness.sentiment_intensity.value_counts().values)

a = train['sadness'].groupby('sentiment_intensity')
over_sadness = pd.DataFrame(a.apply(lambda x: x.sample(MAX_SADNESS, replace=True).reset_index(drop=True))).reset_index(
        drop=True)

train_oversampling = {
    'anger': over_anger,
    'fear': over_fear,
    'joy': over_joy,
    'sadness': over_sadness
}

### Custom Features 

Features personalizadas implementando nuestros propios Transformers (estandar de scikit para crear nuevas features entre otras cosas).




#### Chars Count

In [ ]:
class CharsCountTransformer(BaseEstimator, TransformerMixin):
    def get_relevant_chars(self, tweet):
        num_hashtags = tweet.count('#')
        num_exclamations = tweet.count('!')
        num_interrogations = tweet.count('?')
        num_at = tweet.count('@')
        return [num_hashtags, num_exclamations, num_interrogations, num_at]

    def transform(self, X, y=None):
        chars = []
        for tweet in X:
            chars.append(self.get_relevant_chars(tweet))

        return np.array(chars)

    def fit(self, X, y=None):
        return self

In [ ]:
# Veamos que sucede si ejecutamos el transformer
sample = train['anger'].sample(5, random_state = 1).tweet
sample_features = CharsCountTransformer().transform(sample)

# Se puede verificar que el conteo de símbolos es consistente con el transformer creado.
print(f'Tweet original: {sample}')
print(f'Features creados: {sample_features}')

Tweet original: 604    @everycolorbot more like every color looks the...
527    @thomeagle Just to help maintain and boost our...
894    i live and die for mchanzo honeymoon crashing ...
195    @RealBD_ @ReyesAverie 47 unarmed blacks killed...
422         Drop Snapchat names #bored #snap #swap #pics
Name: tweet, dtype: object
Features creados: [[2 0 0 1]
 [1 0 0 1]
 [0 0 0 0]
 [0 0 0 2]
 [4 0 0 0]]


#### Word Embedding

In [ ]:
############## word embeddings ######################
#tenemos dict de train para esto
emb_cont = {}

for emotion in ['anger', 'fear', 'joy', 'sadness']:
    emb_cont[emotion] = train[emotion].tweet

##codigo del aux 2

from collections import Counter
import string

# limpiar puntuaciones y separar por tokens.
punctuation = string.punctuation + "«»“”‘’…—"
stopwords = pd.read_csv(
    'https://raw.githubusercontent.com/Alir3z4/stop-words/master/english.txt'
).values
stopwords = Counter(stopwords.flatten().tolist())

def simple_tokenizer(doc, lower=False):
    if lower:
        tokenized_doc = doc.translate(str.maketrans(
            '', '', punctuation)).lower().split()

    tokenized_doc = doc.translate(str.maketrans('', '', punctuation)).split()
    tokenized_doc = [
        token for token in tokenized_doc if token.lower() not in stopwords
    ]
    return tokenized_doc

clean_cont = {}

for emotion in ['anger', 'fear', 'joy', 'sadness']:
    clean_cont[emotion] = [simple_tokenizer(doc) for doc in emb_cont[emotion].values]

In [ ]:
#phrases gemsin

phrases = {}

for emotion in ['anger', 'fear', 'joy', 'sadness']:
    phrases[emotion] = Phrases(clean_cont[emotion], min_count=10, progress_per=5000) 

#phraser gemsin
bigram_phrases = {}
sentences = {}

for emotion in ['anger', 'fear', 'joy', 'sadness']:
    bigram_phrases[emotion] = Phraser(phrases[emotion])
    sentences[emotion] = bigram_phrases[emotion][clean_cont[emotion]]

INFO - 23:02:44: collecting all words and their counts
INFO - 23:02:44: PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO - 23:02:44: collected 8012 word types from a corpus of 6447 words (unigram + bigrams) and 941 sentences
INFO - 23:02:44: using 8012 counts as vocab in Phrases<0 vocab, min_count=10, threshold=10.0, max_vocab_size=40000000>
INFO - 23:02:44: collecting all words and their counts
INFO - 23:02:44: PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO - 23:02:44: collected 10696 word types from a corpus of 9099 words (unigram + bigrams) and 1257 sentences
INFO - 23:02:44: using 10696 counts as vocab in Phrases<0 vocab, min_count=10, threshold=10.0, max_vocab_size=40000000>
INFO - 23:02:44: collecting all words and their counts
INFO - 23:02:44: PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO - 23:02:44: collected 7934 word types from a corpus of 6353 words (unigram + bigrams) and 902 sentences
INFO - 23:02:44: using 7934 count

In [ ]:
#ahora, el modelo
import multiprocessing
from time import time 

models = {}
t = time()

for emotion in ['anger', 'fear', 'joy', 'sadness']:
    models[emotion] = Word2Vec(min_count=5,
                      window=4,
                      size=200,
                      sample=6e-5,
                      alpha=0.03,
                      min_alpha=0.0007,
                      negative=20,
                      workers=multiprocessing.cpu_count())
    models[emotion].build_vocab(sentences[emotion], progress_per=10000)
    models[emotion].train(sentences[emotion], 
                          total_examples=models[emotion].corpus_count, epochs=15, 
                          report_delay=10)
    models[emotion].init_sims(replace=True)

INFO - 23:02:47: collecting all words and their counts
INFO - 23:02:47: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 23:02:47: collected 3400 word types from a corpus of 6447 raw words and 941 sentences
INFO - 23:02:47: Loading a fresh vocabulary
INFO - 23:02:47: effective_min_count=5 retains 167 unique words (4% of original 3400, drops 3233)
INFO - 23:02:47: effective_min_count=5 leaves 1755 word corpus (27% of original 6447, drops 4692)
INFO - 23:02:47: deleting the raw counts dictionary of 3400 items
INFO - 23:02:47: sample=6e-05 downsamples 167 most-common words
INFO - 23:02:47: downsampling leaves estimated 182 word corpus (10.4% of prior 1755)
INFO - 23:02:47: estimated required memory for 167 words and 200 dimensions: 350700 bytes
INFO - 23:02:47: resetting layer weights
INFO - 23:02:47: training model with 2 workers on 167 vocabulary and 200 features, using sg=0 hs=0 sample=6e-05 negative=20 window=4
INFO - 23:02:47: worker thread finished; awaiting 

In [ ]:
#feature para words embeddings, código extraído del aux 2
class Doc2VecTransformer(BaseEstimator, TransformerMixin):
    """ Transforma tweets a representaciones vectoriales usando algún modelo de Word Embeddings.
    """
    emotion = ''
    def __init__(self, aggregation_func, emotion):
        # extraemos los embeddings desde el objeto contenedor. ojo con esta parte.
        self.emotion = emotion

        # indicamos la función de agregación (np.min, np.max, np.mean, np.sum, ...)
        self.aggregation_func = aggregation_func

    def simple_tokenizer(self, doc, lower=False):
        """Tokenizador. Elimina signos de puntuación, lleva las letras a minúscula(opcional) y 
           separa el tweet por espacios.
        """
        if lower:
            doc.translate(str.maketrans('', '', string.punctuation)).lower().split()
        return doc.translate(str.maketrans('', '', string.punctuation)).split()
    
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        
        doc_embeddings = []
        
        for doc in X:
            # tokenizamos el documento. Se llevan todos los tokens a minúscula. 
            # ojo con esto, ya que puede que tokens con minúscula y mayúscula tengan
            # distintas representaciones
            tokens = self.simple_tokenizer(doc, lower = True) 
            
            selected_wv = []
            for token in tokens:
                if token in models[emotion].wv.vocab:
                    selected_wv.append(models[emotion].wv[token])
                    
            # si seleccionamos por lo menos un embedding para el tweet, lo agregamos y luego lo añadimos.
            if len(selected_wv) > 0:
                doc_embedding = self.aggregation_func(np.array(selected_wv), axis=0)
                doc_embeddings.append(doc_embedding)
            # si no, añadimos un vector de ceros que represente a ese documento.
            else: 
                #print('No pude encontrar ningún embedding en el tweet: {}. Agregando vector de ceros.'.format(doc))
                doc_embeddings.append(np.zeros(models[emotion].wv.vector_size)) # la dimension del modelo 

        return np.array(doc_embeddings)

#### Emoji Count

In [ ]:
import regex as re
class EmojiCount(BaseEstimator, TransformerMixin):
    def get_relevant_chars(self, tweet):
        split = len(re.findall(r'\U00002764', tweet))
        return [split]

    def transform(self, X, y=None):
        chars = []
        for tweet in X:
            chars.append(self.get_relevant_chars(tweet))
        return np.array(chars)

    def fit(self, X, y=None):
        return self

#### Lexicon

In [ ]:
from lexicons import getLexicons
###################################################################
#Llamamos a getLexicons() de lexicons.py para generar el dict
#completo de lexicons con todas sus variables asociadas.
lexicons_dict = getLexicons()

#retornamos un diccionario que contiene listas con las lexicons y sus
#polaridades para cada sentimiento
def lexicons_sentiments(): 
        emotionsLex = {
            'anger': [],
            'fear': [],
            'joy': [],
            'sadness': []
        }

        for key in lexicons_dict:
            for emotion in ['anger', 'fear', 'joy', 'sadness']:
                if (lexicons_dict[key][4] == ('#' + emotion)):
                    polarity = lexicons_dict[key][6] #positive or negative
                    polarity_val = lexicons_dict[key][7] #polarity value

                    aux_list = [key, polarity, polarity_val]
                    emotionsLex[emotion].append(aux_list)
                    
        return emotionsLex

#Feature para las lexicons de cada tweet
class LexiconCountTransformer(BaseEstimator, TransformerMixin):
    
    #lexicons_dict = {}

    #se ejecuta lexicons_sentiments para asignarle a cada sentimiento
    #estudiado (anger, fear, joy, sadness) las lexicons correspondientes
    lexicons_emotions = {}

    #string aux que almacena el nombre del sentimiento del dataset
    #en proceso de clasificacion (ej: anger)
    name_emotion = ''

    #Al iniciar la clase, se debe llamar al método lexicons_sentiments()
    #para que cree el diccionario correspondiente, a la vez que se asigna
    #el sentimiento del dataset a trabajar.
    
    def __init__(self, nombre_emocion):
        self.name_emotion = nombre_emocion
        self.lexicons_emotions = lexicons_sentiments()

    #entrega promedio polaridad (si es positiva o negativa)
    #y numero de lexicons positivos y negativos
    def get_relevant_chars(self, tweet):
        polarities_values = [0]
        positives = 0
        negatives = 0

        #tweet a lower case para que coincidan las lexicons
        lower_case = tweet.lower()
        
        for lex in self.lexicons_emotions[self.name_emotion]:
            parsed_string = self.parse_underscores(lex[0])
            if (parsed_string in lower_case):
                polarities_values.append(abs(float(lex[2])))

                if (lex[1] == 'positive'):
                    positives += 1
                else:
                    negatives += 1

        return [np.mean(polarities_values), positives, negatives]

    #dado que las keys de las lexicons son precisamente la frase
    #que podría aparecer en el tweet, es necesario parsearlas
    #quitándoles el _ y reemplazándolas por espacios blancos, tal
    #como aparecería en un tweet escrito en lenguaje natural
    def parse_underscores(self, input):
        output = ''
        for char in input:
            if (char == '_'):
                output += ' '
            else:
                output += char
        return output

    def transform(self, X, y=None):
        polarities = []

        for tweet in X:
            polarities.append(self.get_relevant_chars(tweet))

        return np.array(polarities)

    def fit(self, X, y=None):
        return self

#######################################################################


#### Negation Count

In [ ]:
#feature para las negaciones en un documento
from nltk.sentiment.util import mark_negation
class NegationCountTransformer(BaseEstimator, TransformerMixin):

    def get_negated_count(self, tweet):
        count = 0
        negations = mark_negation(tweet.split())

        for word in negations:
            if ("_NEG" in word):
                count +=1
        return [count]

    def transform(self, X, y=None):
        negated = []
        for tweet in X:
            negated.append(self.get_negated_count(tweet))

        return np.array(negated)

    def fit(self, X, y=None):
        return self

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


#### Elongated Word Count

In [ ]:
#feature para las palabras con más de dos letras repetidas
class ElongatedWordsCountTransformer(BaseEstimator, TransformerMixin):

    def get_elongated_words(self, tweet):
        num_elongated = 0
        splits = tweet.split(' ')

        for word in splits:
            if (len(word) > 1):
                i = 0
                while (i < (len(word) - 2)):
                    if (word[i] == word[i+1] == word[i+2]):
                        num_elongated += 1
                        break
                    i+=1
                        
        return [num_elongated]

    def transform(self, X, y=None):
        elongated = []
        for tweet in X:
            elongated.append(self.get_elongated_words(tweet))
        
        return np.array(elongated)

    def fit(self, X, y=None):
        return self

#### Uppercase Count

In [ ]:
#Feature para el número de palabras completamente en mayus
class UppercaseCountTransformer(BaseEstimator, TransformerMixin):

    def get_uppercase_words(self, tweet):
        num_uppercase = 0
        splits = tweet.split(' ')

        for word in splits:
            flag = True

            for char in word:
                if (len(word) == 1):
                    
                    flag = False
                else:
                    if (char.isupper() == False):
                        flag = False
            if (flag == True):
                num_uppercase += 1
        return [num_uppercase]

    def transform(self, X, y=None):
        words_uppercase = []
        for tweet in X:
            words_uppercase.append(self.get_uppercase_words(tweet))
        return np.array(words_uppercase)

    def fit(self, X, y=None):
        return self
 

#### DotsMarks Count

In [ ]:
#feature para secuencias de puntos, exclamaciones e interrogaciones
class DotsMarksCountTransformer(BaseEstimator, TransformerMixin):
    
    #se cuenta si hay al menos tres de cada uno
    def get_dotsmarks_count(self, tweet):
        #number of [dots, ?, !]
        dots = 0
        q_marks = 0
        ex_marks = 0

        splits = tweet.split()

        for word in splits:
            if ('...' in word):
                dots += 1
            if ('???' in word):
                q_marks += 1
            if ('!!!' in word):
                ex_marks += 1

        return [dots, q_marks, ex_marks]

    def transform(self, X, y=None):
        sequences = []
        for tweet in X:
            sequences.append(self.get_dotsmarks_count(tweet))

        return np.array(sequences)

    def fit(self, X, y=None):
        return self

### Custom Tokenizer



Crearemos funciones para cada tokenizador o procesamiento que se desea realizar sobre un tweet.

In [ ]:
# Librerias usadas por algunos tokenizadores
import nltk
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

#### Remover Stopwords

In [ ]:
from nltk.corpus import 
# Tokenizador que remueve stopwords del ingles y simbolos
def remStopWords(tokens):
    newTokens = []
    stopWords = set(stopwords.words('english'))
    stopWords_extended = stopWords | {'.', ',', ':', '+', '-', '(',')',';','\'','..','...','—', '>','<','\\'}
    for token in tokens:
        if (token not in stopWords_extended) and (token[0:1]!='@'):
            newTokens.append(token)
    return newTokens

#### Remover Hashtags

In [ ]:
# Tokenizador que remueve los #
def hashtagToWord(tokens):
    newTokens = []
    for token in tokens:
        newTokens.append(token.replace('#',''))
    return newTokens

#### Lematizar

In [ ]:
from nltk.stem import WordNetLemmatizer
# Funcion que lematiza los terminos
def lemmatize(tokens):
    lem = WordNetLemmatizer()
    lemmatizedTokens = []
    for word in tokens:
        lemmatizedTokens.append(lem.lemmatize(word.lower()))
    return lemmatizedTokens

#### Stemming

In [ ]:
from nltk.stem.snowball import SnowballStemmer
# Funcion que realiza steamming sobre los terminos
def stemmize(tokens):
    ps =  SnowballStemmer(language='english')
    stemmedTokens = []
    for word in tokens:
        stemmedTokens.append(ps.stem(word))
    return stemmedTokens

#### Tokenizador

Creamos una funcion que se encarga de juntar todos los procesamiento y tokkenizadores que deseamos usar.

In [ ]:
from nltk.tokenize import TweetTokenizer
from nltk.sentiment.util import mark_negation
# Funcion que se usara en el Pipeline como tokenizador ,
# la cual agrupa todos los procesamientos y tokenizadores que se usaran sobre
# un tweet.
def my_tokenizer(text):
    tokens = TweetTokenizer(strip_handles=True, reduce_len=True).tokenize(text)
    # Nos dimos cuenta que remover stopwords en un tweet no es favorable.
    # tokens = remStopWords(tokens)
    tokens = mark_negation(tokens)
    tokens = hashtagToWord(tokens)
    tokens = stemmize(tokens)
    tokens = lemmatize(tokens)
    return tokens

Testeo de nuestro tokenizador

In [ ]:
 print(my_tokenizer('Time wounds all heels.\\n\\n #DrunkJesus #rt #lol #wisdom #quote #comedy #self #Revenge  #hate #time #funny #politics #Trump #POTUS2016'))

['time', 'wound', 'all', 'heel', '.', '\\', 'n', '\\', 'n', 'drunkjesus', 'rt', 'lol', 'wisdom', 'quot', 'comedi', 'self', 'reveng', 'hate', 'time', 'funni', 'polit', 'trump', 'potus2016']


In [ ]:
print(my_tokenizer("@soyoprincess they irritate me. Them and their inch thick made up masks"))

['they', 'irrit', 'me', '.', 'them', 'and', 'their', 'inch', 'thick', 'made', 'up', 'mask']


### Definir la representación y el clasificador





In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

Experimento 1

In [ ]:
# LogisticRegression + tfidf + chars_count + negation_count + word_embedding

def experimento1(dataset_name):

    return Pipeline([('features',
                      FeatureUnion([('tfidf_vec', TfidfVectorizer(tokenizer=my_tokenizer, use_idf=True)),
                                    ('chars_count', CharsCountTransformer()),
                                    ('negation_count', NegationCountTransformer()),
                                    ('word_embedding', Doc2VecTransformer(np.mean, dataset_name)),
                                    ])),('clf', LogisticRegression(solver='lbfgs', multi_class='ovr', max_iter = 1000))])

Experimento 2

In [ ]:
# LogisticRegression + bow + chars_count + negation_count + word_embedding

def experimento2(dataset_name):

    return Pipeline([('features',
                      FeatureUnion([('bow', CountVectorizer(tokenizer=my_tokenizer, ngram_range=(1,3), max_features=9000)),
                                    ('chars_count', CharsCountTransformer()),
                                    ('negation_count', NegationCountTransformer()),
                                    ('word_embedding', Doc2VecTransformer(np.mean, dataset_name)),
                                    ])),('clf', LogisticRegression(solver='lbfgs', multi_class='ovr', max_iter = 1000))])

Experimento 3

In [ ]:
# SVC + bow + chars_count + negation_count + word_embedding
 
def experimento3(dataset_name):

    return Pipeline([('features',
                      FeatureUnion([('bow', CountVectorizer(tokenizer=my_tokenizer, ngram_range=(1,3), max_features=9000)),
                                    ('chars_count', CharsCountTransformer()),
                                    ('negation_count', NegationCountTransformer()),
                                    ('word_embedding', Doc2VecTransformer(np.mean, dataset_name)),
                                    ])),('clf', SVC(kernel='linear', probability=True))])

Experimento 4

In [ ]:
# SVC + tfidf + chars_count + negation_count + word_embedding

def experimento4(dataset_name):

    return Pipeline([('features',
                      FeatureUnion([('tfidf_vec', TfidfVectorizer(tokenizer=my_tokenizer, use_idf=True)),
                                    ('chars_count', CharsCountTransformer()),
                                    ('negation_count', NegationCountTransformer()),
                                    ('word_embedding', Doc2VecTransformer(np.mean, dataset_name)),
                                    ])),('clf', SVC(kernel='linear', probability=True))])

Experimento 5

In [ ]:
# Logistic Regression + tfidf + chars_count + negation_count + Dotsmaks_count+ word_embedding

def experimento5(dataset_name):

    return Pipeline([('features',
                      FeatureUnion([('tfidf_vec', TfidfVectorizer(tokenizer=my_tokenizer, use_idf=True)),
                                    ('chars_count', CharsCountTransformer()),
                                    ('negation_count', NegationCountTransformer()),
                                    ('dotsMarks_count', DotsMarksCountTransformer()),
                                    ('word_embedding', Doc2VecTransformer(np.mean, dataset_name))
                                    ])),('clf', LogisticRegression(solver='lbfgs', multi_class='ovr', max_iter = 1000))])

Experimento 6

In [ ]:
# Logistic Regression + tfidf + chars_count + negation_count +  word_embedding + Lexicon

def experimento6(dataset_name):

    return Pipeline([('features',
                      FeatureUnion([('tfidf_vec', TfidfVectorizer(tokenizer=my_tokenizer, use_idf=True)),
                                    ('chars_count', CharsCountTransformer()),
                                    ('negation_count', NegationCountTransformer()),
                                    ('word_embedding', Doc2VecTransformer(np.mean, dataset_name)),
                                    ('lexicon_count', LexiconCountTransformer(dataset_name))
                                    ])),('clf', LogisticRegression(solver='lbfgs', multi_class='ovr', max_iter = 1000))])

### Ejecutar el pipeline para algún dataset

Run principal que deja un 33% de los datos como testeo

In [ ]:
def run(dataset, dataset_name, pipeline):
    """Creamos el pipeline y luego lo ejecutamos el pipeline sobre un dataset. 
    Retorna el modelo ya entrenado mas sus labels asociadas y los scores obtenidos al evaluarlo."""

    # Dividimos el dataset en train y test, aún no se transforma de Strings a valores numéricos.
    X_train, X_test, y_train, y_test = train_test_split(
        dataset.tweet,
        dataset.sentiment_intensity,
        shuffle=True,
        stratify=dataset.sentiment_intensity,
        test_size=0.33)
    
    print(f'# Datos de entrenamiento en dataset {dataset_name}: {len(X_train)}')
    print(f'# Datos de testing en dataset {dataset_name}: {len(X_test)}')

    # Entrenamos el clasificador (Ejecuta el entrenamiento sobre todo el pipeline). 
    # En este caso el Bag of Words es el encargado de transformar de Strings a vectores numéricos.
    pipeline.fit(X_train, y_train)

    # Predecimos las probabilidades de intensidad de cada elemento del set de prueba.
    predicted_probabilities = pipeline.predict_proba(X_test)

    # Obtenemos el orden de las clases aprendidas.
    learned_labels = pipeline.classes_
    
    # Evaluamos:
    scores = evaluate(predicted_probabilities, y_test, learned_labels, dataset_name)
    return pipeline, learned_labels, scores

Este run es mas interesante porque reliza un cross_validation sobre nuestro dataset y nos entrega accuracy, por lo tanto para medir la generalidad nos sirvio mucho mas este run.

In [ ]:
from sklearn.model_selection import cross_val_score
def run2(dataset, dataset_name, pipeline):
    """Creamos el pipeline y luego lo ejecutamos el pipeline sobre un dataset. 
    Retorna el modelo ya entrenado mas sus labels asociadas y los scores obtenidos al evaluarlo."""

    # Dividimos el dataset en train y test, aún no se transforma de Strings a valores numéricos.
    X_train, X_test, y_train, y_test = train_test_split(
        dataset.tweet,
        dataset.sentiment_intensity,
        shuffle=True,
        stratify=dataset.sentiment_intensity,
        test_size=0.33)
    
    # Entrenamos el clasificador (Ejecuta el entrenamiento sobre todo el pipeline). 
    # En este caso el Bag of Words es el encargado de transformar de Strings a vectores numéricos.
    pipeline.fit(X_train, y_train)

    # Predecimos las probabilidades de intensidad de cada elemento del set de prueba.
    predicted_probabilities = pipeline.predict_proba(X_test)

    # Obtenemos el orden de las clases aprendidas.
    learned_labels = pipeline.classes_
    
    # Evaluamos:
    scores = cross_val_score(pipeline, dataset.tweet, dataset.sentiment_intensity, cv=10)
    return pipeline, learned_labels, scores

### Ejecutar el sistema creado por cada train set

Este código crea y entrena los 4 sistemas de clasificación y luego los evalua. Para los experimentos, pueden copiar este código variando el pipeline cuantas veces estimen conveniente.

In [ ]:
classifiers = []
learned_labels_array = []
scores_array = []

# El parametro "a" lo usamos para repetir el experimento varias veces y por lo tanto
# las metricas obtenidas se veian menos afectadas por la aleatoridad en las particiones
a = 1
for i in range(a):
    # Por cada nombre_dataset, dataset en train ('anger', 'fear', 'joy', 'sadness')
    for dataset_name, dataset in train.items():
        
        # creamos el pipeline
        #pipeline = get_experiment_3_pipeline(dataset_name)
        pipeline = experimento1(dataset_name)
        
        # ejecutamos el pipeline sobre el dataset
        classifier, learned_labels, scores = run(dataset, dataset_name, pipeline)

        # guardamos el clasificador entrenado (en realidad es el pipeline ya entrenado...)
        classifiers.append(classifier)

        # guardamos las labels aprendidas por el clasificador
        learned_labels_array.append(learned_labels)

        # guardamos los scores obtenidos
        scores_array.append(scores)

#print("AVG Accuracy:", np.mean(scores_array))
# print avg scores
print(
    "Average scores:\n\n",
    "Average AUC: {0:.3g}\t Average Kappa: {1:.3g}\t Average Accuracy: {2:.3g}"
    .format(*np.array(scores_array).mean(axis=0)))
# print max scores
print(
    "MAX scores:\n\n",
    "MAX AUC: {0:.3g}\t MAX Kappa: {1:.3g}\t MAX Accuracy: {2:.3g}"
    .format(*np.array(scores_array).max(axis=0)))
# print min scores
print(
    "MIN scores:\n\n",
    "MIN AUC: {0:.3g}\t MIN Kappa: {1:.3g}\t MIN Accuracy: {2:.3g}"
    .format(*np.array(scores_array).min(axis=0)))

# Datos de entrenamiento en dataset anger: 630
# Datos de testing en dataset anger: 311
Confusion Matrix for anger:

[[  5  47   1]
 [  8 189   7]
 [  0  38  16]]

Classification Report:

              precision    recall  f1-score   support

         low       0.38      0.09      0.15        53
      medium       0.69      0.93      0.79       204
        high       0.67      0.30      0.41        54

    accuracy                           0.68       311
   macro avg       0.58      0.44      0.45       311
weighted avg       0.63      0.68      0.62       311

# Datos de entrenamiento en dataset fear: 842
# Datos de testing en dataset fear: 415
Confusion Matrix for fear:

[[ 32  59   4]
 [ 31 180  20]
 [  6  53  30]]

Classification Report:

              precision    recall  f1-score   support

         low       0.46      0.34      0.39        95
      medium       0.62      0.78      0.69       231
        high       0.56      0.34      0.42        89

    accuracy                

Resultados de las metricas

In [ ]:
print(scores_array)

[array([0.671, 0.117, 0.678]), array([0.668, 0.168, 0.588]), array([0.748, 0.296, 0.634]), array([0.635, 0.169, 0.546])]


### Predecir los target set y crear la submission

Aquí predecimos los target set usando los clasificadores creados y creamos los archivos de las submissions.

In [ ]:
def predict_target(dataset, classifier, labels):
    # Predecir las probabilidades de intensidad de cada elemento del target set.
    predicted = pd.DataFrame(classifier.predict_proba(dataset.tweet), columns=labels)
    
    # Agregar ids
    predicted['id'] = dataset.id.values
    # Reordenar las columnas
    predicted = predicted[['id', 'low', 'medium', 'high']]
    return predicted

In [ ]:
predicted_target = {}

# Crear carpeta ./predictions
if (not os.path.exists('./predictions')):
    os.mkdir('./predictions')

else:
    # Eliminar predicciones anteriores:
    shutil.rmtree('./predictions')
    os.mkdir('./predictions')

# por cada target set:
for idx, key in enumerate(target):
    # Predecirlo
    predicted_target[key] = predict_target(target[key], classifiers[idx],
                                           learned_labels_array[idx])
    # Guardar predicciones en archivos separados. 
    predicted_target[key].to_csv('./predictions/{}-pred.txt'.format(key),
                                 sep='\t',
                                 header=False,
                                 index=False)

# Crear archivo zip
a = shutil.make_archive('predictions', 'zip', './predictions')

## **7. Resultados**

- Experimentos No.1 y No.2:
Arrojaron resultados promedios casi idénticos, pero poseen algunas diferencias marginalmente menores a causa del Vectorizer elegido. Es difícil comprender el porqué de esto, puede deberse al idf-score que Tfidf posee, ya que considera la relevancia de los términos, mientras que bag of words de CountVectorizer solamente hace un conteo, perdiendo algunos datos.

- Experimento No.3 y No.4:
Es notoriamente mejor el desempeño del No.4. Tiene sentido, ya que Tfidf considera la relevancia de los términos, tal como en el caso anterior. Es curioso ver que, en este caso, el cambio en el Vectorizer utilizado sí arroja diferencias sustanciales usando este modelo, contrastándolo con el caso anterior.

- Experimento No. 5 y No.1:
Añadir la feature de dotsMarks_count mejoró un poco la métrica AUC, pero afectó Kappa. Nuevamente es difícil comprender las razones de esto, con estudios posteriores podría lograr comprenderse.

- Experimento No. 6 y No.1: El No.1 sigue siendo mejor. La feature de Lexicons añadida al No. 6 afectó todas las métricas, en mayor o menor medida. Puede ser que la mayoría de las lexicons no coincidan con el string bruto del tweet, dado que, por una implementación incorrecta, no se analizaba después de ser tokenizado (lo que filtraría hashtags, símbolos, emojis, etc). En caso contrario, puede que mejorara el desempeño.

Se decidió enviar como submission el experimento No.1 a CodaLabs, por su mejor desempeño.


| No. | Approach                       || Dataset   | AUC   | Kappa | Accuracy |
|-----|--------------------------------||-----------|-------|-------|----------|
|     | Features        | Clasifier     |           |       |       |          |
| 0   | bow+chars_count | MultinomialNB | anger     | 0.622 | 0.163 | 0.688    |
|     |                 |               | fear      | 0.597 | 0.091 | 0.559    |
|     |                 |               | joy       | 0.728 | 0.251 | 0.601    |
|     |                 |               | sadness   | 0.645 | 0.166 | 0.581    |
|     |                 |               |**average**| 0.648 | 0.168 | 0.607    |
|||||||
| 1   |tfidfVect + chars_count + NegationCount + WordEmbedding | LogisticRegression  | anger     | 0.604 | 0.061 | 0.646 |
|     |                 |               | fear      | 0.686 | 0.203 | 0.61    |
|     |                 |               | joy       | 0.742 | 0.259 | 0.597    |
|     |                 |               | sadness   | 0.678 | 0.285 | 0.606    |
|     |                 |               |**average**| 0.682 | 0.265 | 0.621    |
|||||||
| 2   |bow (ngram=(1,3), max_features=9000) + chars_count + NegationCount + WordEmbedding | LogisticRegression | anger     | 0.652 | 0.197 | 0.669 |
|     |                 |               | fear      | 0.686 | 0.297 | 0.614    |
|     |                 |               | joy       | 0.711 | 0.279 | 0.597    |
|     |                 |               | sadness   | 0.678 | 0.285 | 0.606    |
|     |                 |               |**average**| 0.682 | 0.265 | 0.621    |
|||||||
| 3   |bow (ngram=(1,3), max_features=9000) + chars_count + NegationCount + WordEmbedding | SVC | anger     | 0.648 | 0.097 | 0.675 |
|     |                 |               | fear      | 0.677 | 0.205 | 0.607    |
|     |                 |               | joy       | 0.689 | 0.243 | 0.601    |
|     |                 |               | sadness   | 0.62 | 0.135 | 0.546    |
|     |                 |               |**average**| 0.659 | 0.17 | 0.607    |
|||||||
| 4   |tfidfVect + chars_count + NegationCount + WordEmbedding | SVC | anger     | 0.685 | 0.177 | 0.685 |
|     |                 |               | fear      | 0.66 | 0.164 | 0.578    |
|     |                 |               | joy       | 0.713 | 0.283 | 0.604    |
|     |                 |               | sadness   | 0.68 | 0.187 | 0.585    |
|     |                 |               |**average**| 0.685 | 0.203 | 0.613    |
|||||||
| 5   |tfidfVect + chars_count + NegationCount + dotsMarks_count + WordEmbedding | LogisticRegression  | anger     | 0.635 | 0.117 | 0.678 |
|     |                 |               | fear      | 0.707 | 0.142 | 0.586    |
|     |                 |               | joy       | 0.72 | 0.258 | 0.607    |
|     |                 |               | sadness   | 0.701 | 0.19 | 0.563    |
|     |                 |               |**average**| 0.687 | 0.194 | 0.621    |
|||||||
| 6   |tfidfVect + chars_count + NegationCount + WordEmbedding + Lexicon | LogisticRegression  | anger     | 0.671 | 0.117 | 0.678 |
|     |                 |               | fear      | 0.668 | 0.168 | 0.588    |
|     |                 |               | joy       | 0.748 | 0.296 | 0.634    |
|     |                 |               | sadness   | 0.635 | 0.169 | 0.546    |
|     |                 |               |**average**| 0.68 | 0.188 | 0.611    |


## **8. Conclusiones**

Un trabajo a futuro es usar un clasificador generalizado para los distintos sentimientos, construyendo pipelines especializados en cada uno. Respecto al tokenizador y features, fue curioso cómo las stopwords empeoraron, en este caso, el desempeño de los clasificadores. En relación a los casos de ensayo, se pudo notar la diferencia entre los modelos, siendo notoria la diferencia entre Naive Bayes con los otros dos. 

La mayor parte de la discusión de resultados está en el apartado anterior, pero detallando un poco más, hay que ser cuidadoso al momento de pre-procesar la información. En este caso, oversampling influyó fuertemente en el overfitting de los clasificadores, siendo contraproducente en el proceso experimental y su análisis en los primeros pasos del experimento.  

Un estudio más profundo de la materia permitiría experimentar con herramientas más complejas y prevenir caer en los problemas mencionados anteriormente, por lo que se apunta a seguir trabajando en esto.